# Automated raster analysis using Google Earth Engine



In [ ]:
pip install earthengine-api -q

In [ ]:
import ee
import geemap

In [ ]:
# Authenticate
ee.Authenticate()  

# Change default project name
ee.Initialize(project='ee-icecap')

In [ ]:
latitude = 66.128
longitude = -50.306
size = 21000

In [ ]:
point = ee.Geometry.Point(longitude, latitude)
region = point.buffer(size/2).bounds()

In [ ]:
Map = geemap.Map()
Map.addLayer(region, {}, "Search Region")
Map.centerObject(region, zoom=10)  # Centre our map on the region of interest
Map

In [ ]:
date_start = '2023-07-01'
date_end = '2023-08-31'

In [ ]:
# Get Landsat 8 image collection
landsat8_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")

# Filter to desired region and date bounds
landsat8_collection = landsat8_collection.filterBounds(region)
landsat8_collection = landsat8_collection.filterDate(date_start, date_end)

In [ ]:
landsat8_collection

In [ ]:
image = landsat8_collection.sort('CLOUD_COVER').first()
image = image.clip(region)

image

In [ ]:
Map = geemap.Map()  # Create empty map

max_reflectance = 0.8 # Set the upper limit of reflectance to visualise.
                       # Play with this value (between 0-1) to see what it
                       # does. It will need to be higher for snowy/icy
                       # scenes.

visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'Colour Composite Image')

Map.centerObject(region, zoom=11)
Map

In [ ]:
# Define name of folder and file
folder = 'ee-icecap'
region_name = 'Kangaamiut-Sermiat'

# Get the data of the image from the metadata
date_string = image.get('DATE_ACQUIRED').getInfo()

# Now we will construct the filename automatically
filename = region_name + '_' + date_string + '_image'

# Visualise for testing
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif')

In [ ]:
import time

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': image.select(['B4', 'B3', 'B2', 'B5', 'B6']),
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

print('Finished processing. Image is exported to your Drive.')

In [ ]:
image.bandNames().getInfo()

In [ ]:
type_of_ndi = 'NDSI'  # type of NDI calculated, for filename purposes
r_high = 'B3'  # relevant band name for r_high
r_low = 'B6'   # relevant band name for r_low

ndi = image.normalizedDifference([r_high, r_low]).rename(type_of_ndi)

ndi

In [ ]:
Map = geemap.Map()

# Display colour image
max_reflectance = 0.80
visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'True Colour')

# Display NDI
visParams = {'bands': [type_of_ndi], 'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}
Map.addLayer(ndi, visParams, type_of_ndi)

# Add colorbar for NDI
Map.add_colorbar(
    vis_params=visParams,
    label="NDSI Index",
    orientation='horizontal',
    position='bottomright'
)

Map.centerObject(region, zoom=11)
Map

## Import shapefile



In [110]:
Map = geemap.Map()
in_shp = "data/RGI2000-v7.0-C-icecap.geojson"

# Convert shapefile to Earth Engine FeatureCollection
polygon_fc = geemap.geojson_to_ee(in_shp)

Map.add_shp(in_shp, layer_name="Ice Cap polygon")

# Add to map with a style
Map.addLayer(polygon_fc, {'color': 'orange'}, 'Ice Cap polygon')

# Center the map on the shapefile
Map.centerObject(polygon_fc, 10)

Map

Could not convert the geojson to ee.Geometry()


Exception: string indices must be integers, not 'str'

In [106]:
pwd

'/Users/jr555/Library/CloudStorage/OneDrive-DukeUniversity/teaching/gds-applications-site/book/lectures/unit3'

In [107]:
cd data

/Users/jr555/Library/CloudStorage/OneDrive-DukeUniversity/teaching/gds-applications-site/book/lectures/unit3/data


In [111]:
# Convert shapefile to Earth Engine FeatureCollection
polygon_fc = geemap.geojson_to_ee(in_shp)

Could not convert the geojson to ee.Geometry()


Exception: string indices must be integers, not 'str'